# CPU 1D Gaussian Filter - Weighted Neighbourhood Operations

This demonstrates how kernels apply different weights to neighbours.

Gaussian filter is a fundamental image processing operation that smooths data while preserving edges better than simple averaging. However to preserve edge, median filter is generally prefered.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Gaussian Function

In [ ]:
def gaussian_func(x, sigma):
    """Formula for Gaussian distribution."""
    coefficient = 1 / (np.sqrt(2 * np.pi) * sigma)
    exponent = -x**2 / (2 * sigma**2)
    return coefficient * np.exp(exponent)

In [ ]:
x_values = np.linspace(-3, 3, 100)
sigma = 1.0
gaussian_values = gaussian_func(x_values, sigma)

plt.figure(figsize=(10, 5))
plt.plot(x_values, gaussian_values, "b-", linewidth=2)
plt.title(r"Gaussian Function: $g(x)=\frac{1}{\sigma\sqrt{2\pi}}\exp\left(\frac{-x^2}{2\sigma^2}\right)$")
plt.xlabel("x (distance from centre)")
plt.ylabel(r"$g(x)$")
plt.grid(True, alpha=0.3)
plt.axvline(x=0, color="red", linestyle="--", alpha=0.5)
plt.axhline(y=0, color="black", linewidth=0.5)

# mark sigma positions
plt.axvline(x=sigma, color="green", linestyle=":", alpha=0.7)
plt.axvline(x=-sigma, color="green", linestyle=":", alpha=0.7)
plt.text(sigma - 0.2, 0.2, r"$\sigma$", fontsize=12, color="green")
plt.text(-sigma + 0.1, 0.2, r"$-\sigma$", fontsize=12, color="green")

plt.tight_layout()
plt.show()

## Compute the Gaussian Weights

In [ ]:
def create_gaussian_kernel(sigma, kernel_size=None):
    """Create 1D Gaussian kernel weights."""
    # input validation
    if sigma <= 0:
        raise ValueError("Sigma must be positive")
    
    # automatically determine kernel size if not provided
    if kernel_size is None:
        # let cover a 6-sigma range
        kernel_size = int(6 * sigma + 1)
        # ensure odd size for symmetry
        if kernel_size % 2 == 0:
            kernel_size += 1
    
    # validate kernel size
    if kernel_size < 1:
        raise ValueError("Kernel size must be at least 1.")
    if kernel_size % 2 == 0:
        raise ValueError("Kernel size must be odd for symmetry.")
    
    # create position array centred at 0
    half_size = kernel_size // 2
    positions = np.arange(-half_size, half_size + 1)
    
    # calculate Gaussian weights (the exponent)
    kernel = np.exp(-(positions**2) / (2 * sigma**2))
    
    # normalise so weights sum to 1
    kernel = kernel / kernel.sum()
    
    return kernel, positions

In [ ]:
def visualise_gaussian_kernel(sigma):
    """Visualise Gaussian kernel weights."""
    kernel, positions = create_gaussian_kernel(sigma)

    plt.figure(figsize=(8, 4))
    plt.bar(positions, kernel, alpha=0.7, color="blue", 
            edgecolor="darkblue")
    plt.title(rf"Gaussian Kernel ($\sigma={sigma}$)")
    plt.xlabel("Position relative to centre")
    plt.ylabel("Weight")
    
    # annotate centre and edge weights
    centre_idx = len(kernel) // 2
    plt.text(0, kernel[centre_idx] - 0.03, 
             f"{kernel[centre_idx]:.3f}", 
             ha="center", fontsize=9)
    
    plt.tight_layout()
    plt.show()

In [ ]:
for sigma in [0.5, 1.0, 2.0]:
    visualise_gaussian_kernel(sigma)

## Gaussian Filter

In [ ]:
def gaussian_filter_1d(data, sigma, boundary_mode="zero", quiet=False):
    """Gaussian filter with different padding strategies."""
    # validate input
    if not isinstance(data, np.ndarray):
        data = np.asarray(data)
    
    if len(data) == 0:
        raise ValueError("Input data cannot be empty")
    
    if boundary_mode not in ("zero", "reflect", "constant"):
        raise ValueError("boundary_mode must be 'zero', 'reflect' or 'constant'")
    
    # create Gaussian kernel
    kernel, positions = create_gaussian_kernel(sigma)
    kernel_size = len(kernel)
    half_size = kernel_size // 2
    
    if not quiet:
        print(f"Gaussian Filter Parameters:")
        print(f"  Sigma: {sigma}")
        print(f"  Kernel size: {kernel_size}")
        print(f"  Centre weight: {kernel[half_size]:.3f}")
        print(f"  Edge weights: {kernel[0]:.3f}, {kernel[-1]:.3f}")
        print(f"  Sum of weights: {kernel.sum():.6f}")
        print(f"  Boundary mode: {boundary_mode}\n")
    
    # prepare output array
    result = np.zeros_like(data)
    
    # each iteration of this outer loop is completely
    # independent - this is why Gaussian filtering is perfect for GPU
    # every thread can process one output element without coordination.
    for i in range(data.size):
        # accumulate weighted sum
        weighted_sum = 0.0
        weight_sum = 0.0
        
        # show calculations for first few elements
        if not quiet and i < 2:
            print(f"Computing position {i}:")
        
        # apply kernel weights
        for k in range(kernel_size):
            # calculate source position
            source_pos = i - half_size + k
            origina_source_pos = source_pos
            
            # handle boundaries based on mode
            if boundary_mode == "zero":
                # zero padding: use 0 for out-of-bound
                if 0 <= source_pos < len(data):
                    value = data[source_pos]
                else:
                    value = 0.0
                weight = kernel[k]
            elif boundary_mode == "reflect":
                # reflect at boundaries
                if source_pos < 0:
                    source_pos = -source_pos
                elif source_pos >= len(data):
                    source_pos = 2 * len(data) - source_pos - 2
                value = data[source_pos]
                weight = kernel[k]
            else:
                # constant
                if source_pos < 0:
                    source_pos = 0
                elif source_pos >= len(data):
                    source_pos = len(data) - 1
                value = data[source_pos]
                weight = kernel[k]

            if weight > 0:
                contribution = value * weight
                weighted_sum += contribution
                weight_sum += weight
                
                if not quiet and i < 2:
                    print(f"  pos {origina_source_pos}: "
                          f"{value:.2f} × {weight:.3f} = "
                          f"{contribution:.3f}")
        
        # normalise by actual weight sum
        if weight_sum > 0:
            result[i] = weighted_sum / weight_sum
        else:
            result[i] = 0
            
        if not quiet and i < 2:
            print(f"  Result: {weighted_sum:.3f} / {weight_sum:.3f} "
                  f"= {result[i]:.3f}\n")
    
    return result

In [ ]:
# noisy signal
x = np.linspace(0, 4*np.pi, 100)
clean_signal = (np.sin(x)**2 - np.cos(x-np.pi/2))*5

noise = np.random.normal(0, 1, len(x))
noisy_signal = clean_signal + noise

In [ ]:
filtered = gaussian_filter_1d(noisy_signal, sigma=2.0)

## Function to Visualise Filtered Signal

In [ ]:
def visualise_gaussian_filter(original, filtered, noisy=None):
    """Visualise the effect of Gaussian filtering."""
    num_plots = 2 if noisy is None else 3
    fig, ax = plt.subplots(num_plots, 1, figsize=(12, 8))
    
    # original signal
    ax[0].plot(original, "b-", label="Original", linewidth=2)
    ax[0].set_title("Original Signal")
    ax[0].set_ylabel("Value")
    ax[0].grid(True, alpha=0.3)
    ax[0].legend()
    
    if noisy is not None:
        ax_filtered = ax[2]
        # noisy signal
        ax[1].plot(noisy, "r-", alpha=0.7, label="Noisy")
        ax[1].plot(original, "b--", alpha=0.5, label="Original")
        ax[1].set_title("Signal with Noise")
        ax[1].set_ylabel("Value")
        ax[1].grid(True, alpha=0.3)
        ax[1].legend()
    else:
        ax_filtered = ax[1]

    # filtered signal
    ax_filtered.plot(filtered, "g-", label="Gaussian Filtered", linewidth=2)
    ax_filtered.plot(original, "b--", alpha=0.5, label="Original")
    ax_filtered.set_title("After Gaussian Filter")
    ax_filtered.set_xlabel("Index")
    ax_filtered.set_ylabel("Value")
    ax_filtered.grid(True, alpha=0.3)
    ax_filtered.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
visualise_gaussian_filter(clean_signal, filtered, noisy=noisy_signal)

## Handling of Edge
Reflect at the edge

In [ ]:
filtered_reflect = gaussian_filter_1d(noisy_signal, sigma=2.0, boundary_mode="reflect", quiet=True)
visualise_gaussian_filter(clean_signal, filtered_reflect, noisy=noisy_signal)

Fill by constant at the edge (the same value at the boundary)

In [ ]:
filtered_constant = gaussian_filter_1d(noisy_signal, sigma=2.0, boundary_mode="constant", quiet=True)
visualise_gaussian_filter(clean_signal, filtered_constant, noisy=noisy_signal)